In [38]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# ТУПАЯ МОДЕЛЬ ЛИНЕЙНОЙ РЕГРЕССИЙ
from sklearn.linear_model import LinearRegression

# Linear Regression, which works by Decision Tree Principals
from sklearn.tree import DecisionTreeRegressor

# Support Vector Machine Regressor, который по идее очень даже мощно работает
from sklearn.svm import SVR


#А теперь мы подгрузим самого мощного отморозка и героя вечера
# На него вся надежда
from sklearn.neural_network import MLPRegressor

# ПРОСТО ПАРАМЕТРЫ ПО КОТОРЫМ МЫ БУДЕМ СУДИТЬ ХОРОШО ЛИ ИЛИ ПЛОХО РАБОТАЕТ НАША МОДЕЛЬКА
from sklearn.metrics import mean_squared_error, r2_score

# Просто для проверки модели для определенных частей полного датасета
from sklearn.model_selection import cross_val_score


# Теперь нужно закачать что то для tuning всех моделей
# Мы примерно выбрали какие модельки будут работать
# Осталось прогонять по моделям теперь,и только
# Поможет нам мощный GridSearch
from sklearn.model_selection import GridSearchCV


In [ ]:
all_data = pd.ExcelFile('task1.xlsx')
data = all_data.parse('TRAIN',index_col="ID")
print(data.shape)
print(data.head())

In [ ]:
data[data.select_dtypes(['object']).columns] = data.select_dtypes(['object']).apply(lambda x: x.astype('category'))
cat_columns = data.select_dtypes(['category']).columns
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)
data = data.fillna(data.mean(), inplace = True)

In [ ]:
# Выбери только одну
# mean normalization
dataframe_main=(data-data.mean())/data.std()
# min-max normalization
dataframe_main=(data-data.min())/(data.max()-data.min())
print(dataframe_main)

In [ ]:
train_data = data[:6000]
test_data = data[6000:]
print(train_data.tail())
print(test_data.head())

In [ ]:
test_data = test_data.drop('ESTIM_COST', axis=1)
print(test_data.head())

In [ ]:
# Здесь просто берем и ебашим весь наш трейнинг датасет в отдельные переменные как X и Y
# TRAIN_VAL название по той причине, потому что для проверки модели мы будем фиттинг делать на трейн парт
# а testing будем делать на вал парт
# ты это знаешь,уеба

# Х ТЕСТ для того чтобы, когда моделька будет готова мы просто возьмем и сделаем ебаный предикт и все
# МАЛ епт
Y_TRAIN_VAL = train_data['ESTIM_COST'].as_matrix()
X_TRAIN_VAL = train_data.drop('ESTIM_COST',axis = 1).as_matrix()
X_TEST = test_data.as_matrix()

In [ ]:
# ТУТ МЫ ПРОСТО ЕБАНУЛИ НА ПАРАМЕТРЫ И ТАРГЕТЫ. 
# TRAIN PART - FOR TRAINING MODEL
# VALIDATION PART - FOR TESTING MODEL IN TUNING PROCESS
# ПОВТОРИЛ ДЛЯ ТЕБЯ УЕБКА
feature_train, feature_val, target_train, target_val = train_test_split( X_TRAIN_VAL,Y_TRAIN_VAL, test_size=0.30)
print(feature_train.shape)
print(feature_val.shape)
print(target_train.shape)
print(target_val.shape)


In [ ]:
# LINEAR REGRESSION MODEL
lrm = LinearRegression()
lrm.fit(feature_train, target_train)
target_predicted = lrm.predict(feature_val)
# Это короче какой то score, который показывает на сколько хорошо мы зафитили и запредиктили данные:
# потом посмотрим что до как
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score
# по этой ссылке пойдешь нахуй короче, потом там посмотришь что за брехня
print('This score show the main accuracy for linear regression {}\n'.format(lrm.score(feature_val,target_val)))


# The coefficients - это короче те ебаные коеф.y = a0 + a1*x1 + a2 * x2 и тд
print('Coefficients: {} \n'.format(lrm.coef_))
print('Intercept: {} \n'.format(lrm.intercept_))

# The mean squared error - че то он пиздец большой бля, хуй его что то докак
print("Root Mean squared error: %.2f"
      % np.sqrt(mean_squared_error(target_val, target_predicted)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(target_val, target_predicted))

In [41]:
dtr =  DecisionTreeRegressor()
dtr.fit(feature_train, target_train)
# dtr_target_predicted = dtr.predict(feature_val)
print("Score Decision Tree Regressor:{}\n".format(dtr.score(feature_val,target_val)))
print("Cross validation score: {} \n".format(cross_val_score(dtr, X_TRAIN_VAL, Y_TRAIN_VAL, cv=20)))

Score Decision Tree Regressor:0.9650560397702651

Cross validation score: [ 0.95360587  0.96504099  0.96903814  0.95806519  0.97864738  0.95173875
  0.96583784  0.96614817  0.97517609  0.94360373  0.95806305  0.9040648
  0.9681612   0.96140223  0.96222641  0.93247561  0.9775708   0.97262666
  0.95473781  0.95080556] 



In [ ]:
svr = SVR()
svr.fit(feature_train, target_train)
# svr_target_predicted = svr.predict(feature_val)
print("Score: {} \n".format(svr.score(feature_val,target_val)))
print("Cross Val Score: {} \n".format(cross_val_score(svr,  X_TRAIN_VAL, Y_TRAIN_VAL, cv=20)))
# И того мы просто понимаем что этот алгоритм просто гавно для нашего датасета
# И мы просто посылаем его нахер

In [ ]:
# Ал енды
# Мы посмотрим как будет работать наш любимый алгоритм, по имени MLPRegressor
# Наконец то нейронные сети, сучки
mlpr = MLPRegressor()
mlpr.fit(feature_train, target_train)
print("Score: {} \n".format(mlpr.score(feature_val,target_val)))
print("Cross Val Score: {} \n".format(cross_val_score(mlpr,  X_TRAIN_VAL, Y_TRAIN_VAL, cv=20)))

In [39]:
dtr_parameters = [{'criterion':['mse','friedman_mse','mae'], 
                   'splitter':['random','best'],
                   'random_state':[5,10,None],
                   'presort':[True, False]}]
dtr_tuned_model = GridSearchCV(DecisionTreeRegressor(),dtr_parameters,cv = 20)
dtr_tuned_model.fit(X_TRAIN_VAL, Y_TRAIN_VAL)
print(dtr_tuned_model.best_score_)
print(dtr_tuned_model.best_params_)

0.963645199036
{'criterion': 'friedman_mse', 'presort': True, 'random_state': None, 'splitter': 'random'}
